In [1]:
import tensorflow as tf
import pickle
import os
import shutil
import tempfile
import datetime
from glob import iglob
import time
# import attention
from collections import deque
import pickle
import pyreader
import numpy as np
import csv
from batchmake import Batcher
import pprint
tf.set_random_seed(777)

In [2]:
datas= 'data_samples/'
vocabulary = 'vocab.csv'
vocabs = []
with open(vocabulary, 'r', newline='', encoding='utf-8') as vocab:
    words = csv.reader(vocab)
    for word in words:
        vocabs.append(word)
print(len(vocabs))

22669


In [3]:
hidden_size=32
sequence=5
embedding_dim=50
attention_size = 50
batch_size=16
vocab_size=len(vocabs)
grucell = tf.nn.rnn_cell.BasicLSTMCell(hidden_size)
X = tf.placeholder(tf.int32, [None, sequence], name='inputs_xdata')
# Y = tf.placeholder(tf.float32, [None, sequence], name='targets_ydata')
Y = tf.placeholder(tf.int32, [None, sequence], name='targets_ydata')
seq_len = tf.placeholder(tf.int32, [None], name='seq_len')

embedding_variable = tf.Variable(tf.random_uniform([vocab_size, embedding_dim],-1.0,1.0), trainable=True)
# inputs_ = tf.transpose(X)
batch_embedded = tf.nn.embedding_lookup(embedding_variable, X)

print(batch_embedded)
with tf.variable_scope('RNN'):
    outputs, states = tf.nn.dynamic_rnn(grucell,
                                        inputs=batch_embedded,
                                        sequence_length=seq_len,
                                        dtype=tf.float32)

Tensor("embedding_lookup:0", shape=(?, 5, 50), dtype=float32)


In [4]:
outputs,states

(<tf.Tensor 'RNN/rnn/transpose:0' shape=(?, 5, 32) dtype=float32>,
 LSTMStateTuple(c=<tf.Tensor 'RNN/rnn/while/Exit_2:0' shape=(?, 32) dtype=float32>, h=<tf.Tensor 'RNN/rnn/while/Exit_3:0' shape=(?, 32) dtype=float32>))

In [5]:
# tf.einsum('')

In [6]:
X_for_fc = tf.reshape(outputs, [-1, hidden_size])
outputs = tf.contrib.layers.fully_connected(X_for_fc, vocab_size, activation_fn=None)

In [7]:
outputs = tf.reshape(outputs, [batch_size, sequence, vocab_size])
outputs

<tf.Tensor 'Reshape_1:0' shape=(16, 5, 22669) dtype=float32>

In [8]:
Y

<tf.Tensor 'targets_ydata:0' shape=(?, 5) dtype=int32>

In [9]:
weights = tf.ones([batch_size, sequence])

sequence_loss = tf.contrib.seq2seq.sequence_loss(
    logits=outputs, targets=Y, weights=weights)
loss = tf.reduce_mean(sequence_loss)


In [14]:
optimizer = tf.train.AdamOptimizer(learning_rate=1e-3).minimize(loss)

In [13]:
file_name_queue = tf.train.string_input_producer(['inputs'])
reader = tf.TFRecordReader()
_, single_x = reader.read(file_name_queue)

context_features = {
    "length": tf.FixedLenFeature([], dtype=tf.int64)
}
sequence_features = {
    "tokens": tf.FixedLenSequenceFeature([], dtype=tf.int64),
    "labels": tf.FixedLenSequenceFeature([], dtype=tf.int64)
}


context_parsed, sequence_parsed = tf.parse_single_sequence_example(serialized=single_x,
                                 context_features=context_features,
                                 sequence_features=sequence_features
                                )
batch_seq, batch_x, batch_y = tf.train.batch(tensors=[context_parsed['length'],sequence_parsed['tokens'],sequence_parsed['labels']],
                                                     batch_size=batch_size, 
                                                     capacity=5000,
                                                     num_threads=1,
                                             dynamic_pad=True
                                                    )

In [15]:
num_epochs = 10
delta = 0.5

Ses = tf.Session()
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=Ses, coord=coord)
Ses.run(tf.global_variables_initializer())

print('start learning!')

for epoch in range(num_epochs):
    
    print('{} start'.format(epoch))
    
    loss_train = 0
    loss_test = 0
    accuracy_train = 0
    accuracy_test =0
    
    
#     for i in range(1000):
    i=0
#     try:
    while True:
        seq_, x_in, y_in = Ses.run([batch_seq, batch_x, batch_y])
        feed_data = {X:x_in, Y:y_in, seq_len:seq_}
#         print(feed_data)
#         lo, acc, opt = Ses.run([loss, accuracy, optimizer], feed_dict=feed_data)
        lo, opt = Ses.run([loss, optimizer], feed_dict=feed_data)
#         accuracy_train+=acc
        loss_train = lo * delta + loss_train * (1-delta)
        i+=1
        if i%100 ==0:
            print(' {} : loss {} acc {}'.format(i, lo, accuracy_train))
#     except:
#         print('batch : ',i)
#         accuracy_train /= 
        

start learning!
0 start
 100 : loss 6.16494607925415 acc 0
 200 : loss 6.080910682678223 acc 0
 300 : loss 4.800240993499756 acc 0
 400 : loss 5.179551124572754 acc 0
 500 : loss 5.307797908782959 acc 0
 600 : loss 4.920200347900391 acc 0
 700 : loss 5.766247749328613 acc 0
 800 : loss 6.080294132232666 acc 0
 900 : loss 4.814537048339844 acc 0
 1000 : loss 5.6472697257995605 acc 0
 1100 : loss 5.105912208557129 acc 0
 1200 : loss 5.140685081481934 acc 0
 1300 : loss 3.7921271324157715 acc 0
 1400 : loss 6.111734867095947 acc 0
 1500 : loss 4.88701057434082 acc 0
 1600 : loss 4.413029193878174 acc 0
 1700 : loss 4.571654319763184 acc 0
 1800 : loss 5.0042314529418945 acc 0
 1900 : loss 4.040004730224609 acc 0
 2000 : loss 4.054974555969238 acc 0
 2100 : loss 4.527651786804199 acc 0
 2200 : loss 4.9951043128967285 acc 0
 2300 : loss 4.15425968170166 acc 0
 2400 : loss 6.330731391906738 acc 0
 2500 : loss 4.727242946624756 acc 0
 2600 : loss 2.779005527496338 acc 0
 2700 : loss 4.1598072

ValueError: Cannot feed value of shape (16, 0) for Tensor 'inputs_xdata:0', which has shape '(?, 5)'

In [32]:
# feed_data = {X:}
# opt = Ses.run([outputs], feed_dict=feed_data)
# print(opt[0])
# feed_data
print(batch_seq)
seq_, x_in, y_in = Ses.run([batch_seq, batch_x, batch_y])
print(x_in, y_in)
if len(x_in) != 0:
    print('dd')

Tensor("batch:0", shape=(16,), dtype=int64)
[] []
dd


In [ ]:
predict = tf.nn.top_k(outputs,5)

In [15]:
seq_, x_in, y_in = Ses.run([batch_seq, batch_x, batch_y])
feed_data = {X:x_in, seq_len:seq_, Y:y_in}
pr = Ses.run(predict, feed_dict=feed_data)
feed_data

NameError: name 'predict' is not defined

In [83]:
prob = pr[0]
probidx = pr[1]
idx_to_word = []
with open(vocabulary, 'r', newline='', encoding='utf-8') as vocab:
    words = csv.reader(vocab)
    for word in words:
        idx_to_word.append(word)

In [100]:
probidx
print(feed_data[X], feed_data[Y])
for line in probidx:
    for idx in line:
#         for each_idx in idx:
        print(idx_to_word[idx[0]], idx[0])
    print()
# predic_word = [idx_to_word[i] for i in probidx[0]]

[[6282    4    8    2    9]
 [5482    7   15    5   37]
 [   5  186    3    3    1]
 [  23    3 1216    6  943]
 [   4    4    6   31    4]
 [   2   16    3 5482   25]
 [  31    4    8    2    9]
 [  15    5   37    5   41]
 [   3    3    1   23    3]
 [ 482    6 1216    6  413]
 [   4    4    6   15    5]
 [  72    4    2   18    2]
 [  10   10   15    5   80]
 [   5   84    5  144    3]
 [   4    2  294    7    3]
 [   1   23    3 1216    6]] [[   4    8    2    9 5482]
 [   7   15    5   37    5]
 [ 186    3    3    1   23]
 [   3 1216    6  943    4]
 [   4    6   31    4    2]
 [  16    3 5482   25   31]
 [   4    8    2    9   15]
 [   5   37    5   41    3]
 [   3    1   23    3  482]
 [   6 1216    6  413    4]
 [   4    6   15    5   72]
 [   4    2   18    2   10]
 [  10   15    5   80    5]
 [  84    5  144    3    4]
 [   2  294    7    3    1]
 [  23    3 1216    6  482]]
['from'] 9
['if'] 3
['self'] 1
['0'] 5
['if'] 3

['if'] 3
['if'] 3
['0'] 5
['if'] 3
['if'] 3

['self']